In [81]:
rm(list = ls())

In [82]:
# libraries
suppressMessages({
  library(tidyverse)
  library(broom)
  library(ranger)
}
)

In [83]:
Data <- read.csv("/Users/mpaga/Downloads/train.csv",sep=",")

In [84]:
dim(Data)

[1] 1460   81

In [85]:
names(Data)

 [1] "Id"            "MSSubClass"    "MSZoning"      "LotFrontage"   "LotArea"      
 [6] "Street"        "Alley"         "LotShape"      "LandContour"   "Utilities"    
[11] "LotConfig"     "LandSlope"     "Neighborhood"  "Condition1"    "Condition2"   
[16] "BldgType"      "HouseStyle"    "OverallQual"   "OverallCond"   "YearBuilt"    
[21] "YearRemodAdd"  "RoofStyle"     "RoofMatl"      "Exterior1st"   "Exterior2nd"  
[26] "MasVnrType"    "MasVnrArea"    "ExterQual"     "ExterCond"     "Foundation"   
[31] "BsmtQual"      "BsmtCond"      "BsmtExposure"  "BsmtFinType1"  "BsmtFinSF1"   
[36] "BsmtFinType2"  "BsmtFinSF2"    "BsmtUnfSF"     "TotalBsmtSF"   "Heating"      
[41] "HeatingQC"     "CentralAir"    "Electrical"    "X1stFlrSF"     "X2ndFlrSF"    
[46] "LowQualFinSF"  "GrLivArea"     "BsmtFullBath"  "BsmtHalfBath"  "FullBath"     
[51] "HalfBath"      "BedroomAbvGr"  "KitchenAbvGr"  "KitchenQual"   "TotRmsAbvGrd" 
[56] "Functional"    "Fireplaces"    "FireplaceQu"   "GarageType"

In [86]:
#remove Id colum
Data["Id"] <- NULL
dim(Data)

[1] 1460   80

In [87]:
# numeric data
numData <- Data |> 
  select_if(is.numeric) 

In [88]:
hist(Data$SalePrice)

In [89]:
summary(Data$SalePrice)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  34900  129975  163000  180921  214000  755000 

In [90]:
skimr::skim_without_charts(Data,where(is.numeric))-> num_skimData

In [91]:
(numVal_na <- num_skimData$skim_variable[num_skimData$n_missing !=0] )

[1] "LotFrontage" "MasVnrArea"  "GarageYrBlt"

In [92]:
skimr::skim_without_charts(Data,where(is.character))-> cat_skimData

In [93]:
summary(cat_skimData)

── Data Summary ────────────────────────
                           Values
Name                       Data  
Number of rows             1460  
Number of columns          80    
_______________________          
Column type frequency:           
  character                43    
________________________         
Group variables            None  

Let's impute some of these numerical features

In [94]:
colMeans(is.na(Data[numVal_na]))

LotFrontage  MasVnrArea GarageYrBlt 
0.177397260 0.005479452 0.055479452 

In [95]:
# list of numeric features to impute
imputeVal_list <- apply(Data[numVal_na],2, FUN = "median",na.rm = T,simplify = list)

In [96]:
#impute numerical features 
Data[numVal_na] <- replace_na(Data[numVal_na] ,
                              replace = imputeVal_list
)

In [97]:
# check  na
colMeans(is.na(Data[numVal_na]))

LotFrontage  MasVnrArea GarageYrBlt 
          0           0           0 

In [98]:
# duplicates rows
Data |> 
  select_if(is.numeric) |> 
  unique() |> 
  nrow()

[1] 1460

In [99]:
# select numeric features
Data |> 
  select_if(is.numeric)  ->numData 

# check relationship with target (linear model)
# h0 : there no relationship between X and Y(SalePrice)
# h1 :otherwise

  lm(SalePrice~.,numData) |> 
    summary() |> 
    tidy() |> 
    filter(p.value <=0.5) |> 
    nrow()

[1] 28

28 numerical features have predictive effect on target feature. 

In [100]:
# correlation in numeric features
for (feature in names(numData)[-1]){
  if (cor(numData[names(numData)][1],numData[feature]) >= 0.8 ) print(feature)
}

There is no carrelated features in numeric features 

cat var feat engineering  

In [101]:
# categorical features presenting missing values 
sum(cat_skimData$n_missing  > 0)

[1] 16

In [102]:
# n_missing proportion > 0.95 in cat features 
cat_rid <- cat_skimData$skim_variable[cat_skimData$n_missing >= 0.95] 
length(cat_rid)

[1] 16

16 features present missing values or NA proportion above 0.95, we could get rid of it  

In [103]:
# remove these categorical features
Data[,cat_rid]<- NULL

In [104]:
# check categorical features with NA in Data 
sum(cat_rid %in% names(Data))
rm(cat_rid)

In [105]:
# new data dimension
dim(Data)

[1] 1460   64

In [106]:
# NA check
colMeans(is.na(Data)) |> sum()

[1] 0

There is no longer NA in this dataset

to be continued 😉!